# Quizz Video Generator

## Introduction

In [1]:
"""Experimental module for subtitles support."""

import re

import numpy as np

from moviepy.decorators import convert_path_to_string
from moviepy.tools import convert_to_seconds
from moviepy.video.VideoClip import TextClip, VideoClip


class SubtitlesClipMine(VideoClip):
    """A Clip that serves as "subtitle track" in videos.

    One particularity of this class is that the images of the
    subtitle texts are not generated beforehand, but only if
    needed.

    Parameters
    ----------

    subtitles
        Either the name of a file as a string or path-like object, or a list

    font
        Path to a font file to be used. Optional if make_textclip is provided.

    make_textclip
        A custom function to use for text clip generation. If None, a TextClip
        will be generated.

        The function must take a text as argument and return a VideoClip
        to be used as caption

    encoding
        Optional, specifies srt file encoding.
        Any standard Python encoding is allowed (listed at
        https://docs.python.org/3.8/library/codecs.html#standard-encodings)

    Examples
    --------

    .. code:: python

        from moviepy.video.tools.subtitles import SubtitlesClip
        from moviepy.video.io.VideoFileClip import VideoFileClip
        generator = lambda text: TextClip(text, font='./path/to/font.ttf',
                                        font_size=24, color='white')
        sub = SubtitlesClip("subtitles.srt", make_textclip=generator, encoding='utf-8')
        myvideo = VideoFileClip("myvideo.avi")
        final = CompositeVideoClip([clip, subtitles])
        final.write_videofile("final.mp4", fps=myvideo.fps)

    """

    def __init__(self, subtitles, font=None, make_textclip=None, encoding=None):
        VideoClip.__init__(self, has_constant_size=False)

        if not isinstance(subtitles, list):
            # `subtitles` is a string or path-like object
            subtitles = file_to_subtitles(subtitles, encoding=encoding)

        # subtitles = [(map(convert_to_seconds, times), text)
        #              for times, text in subtitles]
        self.subtitles = subtitles
        self.textclips = dict()

        self.font = font

        if make_textclip is None:
            if self.font is None:
                raise ValueError("Argument font is required if make_textclip is None.")
            
            def make_textclip(txt):
                txt_clip = TextClip(text=txt, font=self.font, font_size=70, size=(700, None), color='white',method='caption',text_align="center",stroke_color="black",stroke_width=10)  
                #txt_clip = txt_clip.with_position(("center", "bottom"))
                return txt_clip

        self.size = (1080,None)  # Modiff
        self.make_textclip = make_textclip
        self.start = 0
        self.duration = max([tb for ((ta, tb), txt) in self.subtitles])
        self.end = self.duration
        

        def add_textclip_if_none(t):
            """Will generate a textclip if it hasn't been generated asked
            to generate it yet. If there is no subtitle to show at t, return
            false.
            """
            sub = [
                ((text_start, text_end), text)
                for ((text_start, text_end), text) in self.textclips.keys()
                if (text_start <= t < text_end)
            ]
            if not sub:
                sub = [
                    ((text_start, text_end), text)
                    for ((text_start, text_end), text) in self.subtitles
                    if (text_start <= t < text_end)
                ]
                if not sub:
                    return False
            sub = sub[0]
            if sub not in self.textclips.keys():
                self.textclips[sub] = self.make_textclip(sub[1])

            return sub

        def frame_function(t):
            sub = add_textclip_if_none(t)
            return self.textclips[sub].get_frame(t) if sub else np.array([[[0, 0, 0]]])

        def make_mask_frame(t):
            sub = add_textclip_if_none(t)
            return self.textclips[sub].mask.get_frame(t) if sub else np.array([[0]])

        self.frame_function = frame_function
        hasmask = bool(self.make_textclip("T").mask)
        self.mask = VideoClip(make_mask_frame, is_mask=True) if hasmask else None



@convert_path_to_string("filename")
def file_to_subtitles(filename, encoding=None):
    """Converts a srt file into subtitles.

    The returned list is of the form ``[((start_time,end_time),'some text'),...]``
    and can be fed to SubtitlesClip.

    Only works for '.srt' format for the moment.
    """
    times_texts = []
    current_times = None
    current_text = ""
    with open(filename, "r", encoding=encoding) as file:
        for line in file:
            times = re.findall("([0-9]*:[0-9]*:[0-9]*,[0-9]*)", line)
            if times:
                current_times = [convert_to_seconds(t) for t in times]
            elif line.strip() == "":
                times_texts.append((current_times, current_text.strip("\n")))
                current_times, current_text = None, ""
            elif current_times:
                current_text += line
    return times_texts

### Imports

In [2]:
import os
import json
import glob
import random
from datetime import timedelta
from PIL import Image

from classes.APIs import *
from classes.utils import Speech

## Generate questions

In [3]:
#print(generate_text("hot-air balloon"))

# Create class Project

In [4]:
from moviepy import CompositeVideoClip,VideoFileClip,VideoClip,vfx # Vidéo
from moviepy.video.tools.subtitles import SubtitlesClip # Sous Titres
from moviepy import ImageClip,TextClip # Image
from moviepy import vfx,AudioFileClip,concatenate_audioclips,CompositeAudioClip # Audio
from moviepy.video.fx import MultiplySpeed
from moviepy.audio.fx import MultiplyVolume



class Project():
    def __init__(self, project_name,speech, subject = None, output_path=None):
        self.project_name = project_name
        self.subject = subject if subject else project_name
        self.output_path = output_path if output_path else os.path.join("outputs", project_name)
        self.paths = self.create_dirs()
        self.speech = speech
        self.create_hashtags()
    def create_dirs(self):
        
        if os.path.exists(self.output_path):
            return
        os.makedirs(self.output_path, exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "audios_Mike"), exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "audios_Eva"), exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "final"), exist_ok=True)
        os.makedirs(os.path.join(self.output_path, "background"), exist_ok=True)
        return {
            "default": self.output_path,
            "audios_Mike": os.path.join(self.output_path, "audios_Mike"),
            "audios_Eva": os.path.join(self.output_path, "audios_Eva"),
            "final": os.path.join(self.output_path, "final"),
            "background": os.path.join(self.output_path, "background")
        }
    def create_hashtags(self):
        # Write the hashtags to a .txt file
        with open(os.path.join(self.paths["final"], "hashtags.txt"), "w") as file:
            file.write(self.speech.text["hashtags"])

        print("File 'hashtags.txt' has been created!")
    
    def random_music(self):
        # Répertoire contenant les fichiers audio .mp3
        audio_dir = "..\musique"  # Remplace par le chemin de ton répertoire

        # Liste tous les fichiers dans le répertoire audio et garde seulement ceux avec l'extension .mp3
        audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.mp3')]

        # Vérifie s'il y a des fichiers .mp3 dans le répertoire
        if audio_files:
            # Choisit un fichier audio au hasard
            random_audio_file = random.choice(audio_files)
            
            # Crée le chemin complet du fichier audio choisi
            audio_path = os.path.join(audio_dir, random_audio_file)
            
            # Charge l'audio dans MoviePy
            musique = AudioFileClip(audio_path)

            print(f"\nFichier audio choisi : {random_audio_file}\n")
            return musique
            
        else:
            print("Aucun fichier audio .mp3 trouvé dans le répertoire.")
            return None
        
    def random_video(self):
        # Répertoire contenant les fichiers vidéo .mp4
        video_dir = "../video_satisfaisante"  # Remplace par le chemin de ton répertoire

        # Liste tous les fichiers dans le répertoire vidéo et garde seulement ceux avec l'extension .mp4
        video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]

        # Vérifie s'il y a des fichiers .mp4 dans le répertoire
        if video_files:
            # Choisit un fichier vidéo au hasard
            random_video_file = random.choice(video_files)
            
            # Crée le chemin complet du fichier vidéo choisi
            video_path = os.path.join(video_dir, random_video_file)
            
            # Charge la vidéo dans MoviePy
            video_clip = VideoFileClip(video_path)

            print(f"\nFichier vidéo choisi : {random_video_file}\n")
            return video_clip
        else:
            print("\nAucune vidéo MP4 trouvée dans le répertoire spécifié.\n")
            return None

    def create_srt(self,Character, duration=5, min_duration=1, max_length=50):
        """
        Crée un fichier .srt à partir d'un texte, en divisant par les ponctuations ".", "!", "?" et
        en calculant la durée des timestamps en fonction du nombre de caractères par phrase.
        
        :param duration: Durée totale de la vidéo en secondes.
        :param min_duration: Durée minimale d'un sous-titre (en secondes).
        :param max_length: Longueur maximale d'une phrase avant de la couper (en caractères).
        """
        def split_long_line(line, max_length):
            """
            Découpe une phrase en plusieurs morceaux, chacun respectant la longueur maximale.
            La coupure se fait au niveau des espaces pour ne pas couper les mots.
            """
            if len(line) <= max_length:
                return [line]
            
            words = line.split()
            chunks, current_chunk = [], ""

            for word in words:
                # Si le mot ajouté dépasse la longueur max, stocker le chunk actuel et recommencer un nouveau
                if len(current_chunk) + len(word) + 1 > max_length:  # +1 pour l'espace
                    chunks.append(current_chunk.strip())
                    current_chunk = word  # Commencer une nouvelle ligne avec le mot actuel
                else:
                    current_chunk += (" " + word if current_chunk else word)
            
            # Ajouter le dernier chunk s'il reste du texte
            if current_chunk:
                chunks.append(current_chunk.strip())
            
            return chunks

        chemin = ""
        if(Character=="Eva"):
            text = self.speech.Eva_view
            chemin = os.path.join(self.paths["audios_Eva"], "subtitles.srt")
        else:
            text = self.speech.Mike_view
            chemin = os.path.join(self.paths["audios_Mike"], "subtitles.srt")

        # Diviser le texte en phrases avec ".", "!" et "?"
        lines = re.split(r'(?<=[.!?])\s+', text)  # Garde le séparateur, puis divise sur l'espace suivant
        lines = [line.strip() for line in lines if line.strip()]  # Nettoyer et enlever les phrases vides

        # Découper les phrases trop longues en plusieurs morceaux
        split_lines = []
        for line in lines:
            split_lines.extend(split_long_line(line, max_length))

        # Calculer le nombre total de caractères dans toutes les phrases
        total_characters = sum(len(line) for line in split_lines)

        
        # Ouvrir le fichier de sortie en mode écriture
        with open(chemin, "w", encoding="utf-8") as f:
            current_time = 0  # Timestamp de début pour la première phrase

            for i, line in enumerate(split_lines):
                # Calculer la durée de chaque segment en fonction du nombre de caractères
                segment_duration = max((len(line) / total_characters) * duration, min_duration)

                # Calculer le timestamp de début et de fin pour chaque ligne
                start_time = str(timedelta(seconds=current_time))
                end_time = str(timedelta(seconds=current_time + segment_duration))

                # Formatter les timestamps au format SRT (hh:mm:ss,ms)
                start_time = start_time.split(".")[0] + ",000"
                end_time = end_time.split(".")[0] + ",000"

                # Écrire dans le fichier au format SRT
                f.write(f"{i + 1}\n")  # Numéro du sous-titre
                f.write(f"{start_time} --> {end_time}\n")  # Timestamp
                f.write(f"{line.strip()}\n\n")  # Contenu du sous-titre

                # Mettre à jour le temps courant
                current_time += segment_duration

                
    def combined_audio(self,chemin):
        
        # Ici je change je prende que l'audio
        # Ne pas oublier de mettre .mp3 si OpenAI
        concatenate_audio = AudioFileClip(os.path.join(chemin, "history.mp4"))

        # Augmenter la vitesse de l'audio final (1.2x)
        #concatenate_audio = MultiplySpeed(factor=1.2).apply(concatenate_audio)

        musique = self.random_music()
        if musique is None:
            return None
        musique = musique.subclipped(0,concatenate_audio.duration)

        # gérer le son pour date
        musique_faible = musique.with_effects([MultiplyVolume(0.0125)]) # réduire le volume de la musique à 2,5 %

        # Superposer les deux audios
        combined_audio = CompositeAudioClip([concatenate_audio, musique_faible])      
        
        # Exporter l'audio combiné
        combined_audio.write_audiofile(os.path.join(chemin, "combined_audio.mp3"))

        return combined_audio
    
    def combined_video(self,Character,chemin,duration=5): 

        # Charger la vidéo
        video_clip = self.random_video()
        if(video_clip is None):
            return None
        
        start_video = random.uniform(0, video_clip.duration - duration - 1)

        
        video_clip=video_clip.subclipped(start_video,start_video+duration)
        print(f"La vidéo commence à  {start_video} s")
        
        # Charger l'avatar
        avatar_clip = VideoFileClip(os.path.join(chemin, "history.mp4"))
        
        avatar_clip=avatar_clip.with_effects([vfx.Crop(x1= 180,y1 =0,x2= 900,y2=720)])
        avatar_clip=avatar_clip.with_effects([vfx.Resize((1080, 1080))])       
        # Redimensionner la vidéo pour être à la bonne taille
        video_clip=video_clip.with_effects([vfx.Resize((1080, 840))])
        
        avatar_clip = avatar_clip.with_position(("center", "top"))
        video_clip = video_clip.with_position(("center", "bottom"))

        # creer le dossier srt subtitles
        self.create_srt(Character,duration)
        subtitles = self.combined_subtitles(chemin,duration)

        # Combiner les deux clips
        final_clip = CompositeVideoClip([avatar_clip, video_clip, subtitles], size=(video_clip.w, video_clip.h + avatar_clip.h))

        #final_clip.write_videofile(os.path.join(self.paths["final"], "output_video_test.mp4"), codec='libx264', fps=24)
        
        return final_clip 
    
    def combined_subtitles(self,chemin,duration=5):
        subtitles = SubtitlesClipMine(subtitles=os.path.join(chemin, "subtitles.srt"), font="fonts\Super Cartoon.ttf", encoding="utf-8")
        subtitles = subtitles.with_position(("center", 800))
        return subtitles
    
    def combined_text(self,Character,duration=5):
        # Create a text clip
        txt_clip:TextClip = TextClip(text=f"{Character}'s point of view ", color='white',font="fonts\Super Cartoon.ttf", font_size=70,size=(1080, None))
        txt_clip = txt_clip.with_position(("center", "top")).with_duration(duration-2).with_start(1)

        txt_clip = txt_clip.with_effects([vfx.FadeIn(1, initial_color=[0, 0, 0]),
                                        vfx.FadeOut(1, final_color=[0, 0, 0])])
        
        txt_clip = txt_clip.with_mask()

        return txt_clip
    
    def generate_video_final(self):
        
        Characters = ["Eva","Mike"]

        for Character in Characters :
            if(Character == "Eva"):
                chemin =  self.paths["audios_Eva"]
            else:
                chemin =  self.paths["audios_Mike"]
            # Charger les fichiers audio
            audio = self.combined_audio(chemin)
            if audio is None:
                print("pas de musique")
                return None
            
            # charger la vidéo
            final_clip = self.combined_video(Character,chemin,audio.duration)
            if(final_clip is None):
                print("pb video")
                return None
            
            # charger le text
            txt_clip = self.combined_text(Character,audio.duration)       
            
            # Ajouter l'audio combiné à la vidéo
            final_clip.audio = audio 
            # Ajouter le texte
            final_clip = CompositeVideoClip([final_clip, txt_clip])

            final_clip.write_videofile(os.path.join(self.paths["final"], f"output_video_{Character}.mp4"), codec='libx264', fps=24)


    
        

    
    def __str__(self):
        return f"Project about {self.subject}"
    

<>:43: SyntaxWarning: invalid escape sequence '\m'
<>:231: SyntaxWarning: invalid escape sequence '\S'
<>:237: SyntaxWarning: invalid escape sequence '\S'
<>:43: SyntaxWarning: invalid escape sequence '\m'
<>:231: SyntaxWarning: invalid escape sequence '\S'
<>:237: SyntaxWarning: invalid escape sequence '\S'
C:\Users\DELL\AppData\Local\Temp\ipykernel_2324\2733548615.py:43: SyntaxWarning: invalid escape sequence '\m'
  audio_dir = "..\musique"  # Remplace par le chemin de ton répertoire
C:\Users\DELL\AppData\Local\Temp\ipykernel_2324\2733548615.py:231: SyntaxWarning: invalid escape sequence '\S'
  subtitles = SubtitlesClipMine(subtitles=os.path.join(chemin, "subtitles.srt"), font="fonts\Super Cartoon.ttf", encoding="utf-8")
C:\Users\DELL\AppData\Local\Temp\ipykernel_2324\2733548615.py:237: SyntaxWarning: invalid escape sequence '\S'
  txt_clip:TextClip = TextClip(text=f"{Character}'s point of view ", color='white',font="fonts\Super Cartoon.ttf", font_size=70,size=(1080, None))


## Video

In [5]:
def generate_audios(speech:Speech, output_audio_path:str):
    voice=create_audio(speech.history, os.path.join(output_audio_path, "history.mp3"))
    print("\nVoice used : "+voice+"\n")
        

In [6]:
def generate_background_google(subject:str, output_background_path:str):
    url_return = create_background_google(subject,output_background_path)
    png_files = glob.glob(os.path.join(output_background_path, "*.png"))
    square_png_files = []
    smallest_diff = 0
    compteur=0
    for file_path in png_files:
        with Image.open(file_path) as img:
            # Obtenir les dimensions de l'image
            width, height = img.size

            # Vérifier si l'image est carrée (largeur == hauteur)
            if width == height:
                # Ajouter le chemin de l'image carrée à la liste
                square_png_files.append(file_path)
                return url_return[compteur]
            
            else:
                # Calculer la différence entre largeur et hauteur
                diff = width/height
                
                # Si l'image est plus proche d'un carré, on la garde
                if abs(diff - 1) < abs(smallest_diff - 1):  # Vérifie la proximité avec 1
                    smallest_diff = diff
                    closest_image = url_return[compteur]
        compteur+=1

    return closest_image

In [7]:
def generate_avatar(nb,speech:Speech,Character, output_audio_path:str, url:str):
    if(Character=="Eva"):
        creer_avatar(nb,speech.Eva_view,os.path.join(output_audio_path, "history.mp4"),url,Character)
    else:
        creer_avatar(nb,speech.Mike_view,os.path.join(output_audio_path, "history.mp4"),url,Character)

## Générer que l'audio avec les dirs 

In [8]:
"""
subject = "Freeze Corleone"
project = Project(subject)
speech = Speech(subject)
speech.print_speech()

generate_audios(speech,project.paths["audios"])
project.combined_audio()
"""

'\nsubject = "Freeze Corleone"\nproject = Project(subject)\nspeech = Speech(subject)\nspeech.print_speech()\n\ngenerate_audios(speech,project.paths["audios"])\nproject.combined_audio()\n'

## Générer que la vidéo avec les dirs

In [9]:
"""
subject = "Blaize Matuidi"
project = Project(subject)
speech = Speech(subject)
print(speech)

generate_background_google(subject,project.paths["background"])
project.combined_video(5)
"""



'\nsubject = "Blaize Matuidi"\nproject = Project(subject)\nspeech = Speech(subject)\nprint(speech)\n\ngenerate_background_google(subject,project.paths["background"])\nproject.combined_video(5)\n'

## Générer toute la vidéo 

In [ ]:

subjects = [
    "arcade",
    "bowling alley",
    "mini golf",
    "escape room",
    "theater play",
    "concert",
    "cooking class",
    "dance class",
    "karaoke bar",
    "camping trip",
    "stargazing",
    "boat ride",
    "ski trip",
    "road trip",
    "hot air balloon ride",
    "comedy club",
    "theme park",
    "botanical garden",
    "farmer's market",
    "ice skating rink",
    "drive-in movie",
    "sports game",
    "wine tasting",
    "brewery tour",
    "pottery class",
    "bookstore date",
    "science center",
    "thrift shopping",
    "DIY craft night",


nb=10
for subject in subjects :
    speech = Speech(subject,nb)
    print(speech)
    project = Project(subject,speech)
    url_choisit=generate_background_google(subject,project.paths["background"])
    generate_avatar(nb,speech,"Eva",project.paths["audios_Eva"],url_choisit)
    generate_avatar(nb,speech,"Mike",project.paths["audios_Mike"],url_choisit)
    project.generate_video_final()
    nb+=1





Speech about 'arcade':
Full Mike's History: Okay, let me tell you about date #8 with Eva! We hit the arcade, and I'm feeling hyped! 🎉 I mean, what better way to impress your date than wreck her in Mario Kart? So, we start the race, and I play it cool, right? But then, outta nowhere, she yells, ‘Eat my dust, Mike!’ and I’m like, 0.1% nervous and 99.9% determined! 💪 She bumps me trying to distract me, but it’s all good—I’m focused! I zoom past her like I’m in a Fast & Furious movie. 🚗💨 Then, her ultimate strategy? Challenge me to Dance Dance Revolution! I swear, I’ve never seen someone move like an octopus on roller skates! 😂 I’m thinking this is easy, right? But she keeps missing steps and laughing so hard I almost fall off the platform! Let’s just say, she’s lucky she’s cute because I was ready to run! If you wanna check out Eva’s epic dance fails, make sure to hit that follow! 💥
Full Eva's History: Alright, y'all, date #8 with Mike—so I thought we’d hit up the arcade to find out if he